In [ ]:
import torch

a = torch.tensor([ range(i,i+3) for i in [0,3,6]])
a

tensor([[0, 1, 2],
        [3, 4, 5],
        [6, 7, 8]])

In [ ]:
a.storage()
# a.untyped_storage()

<ipython-input-2-07cd75b5fe08>:1: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  a.storage()


 0
 1
 2
 3
 4
 5
 6
 7
 8
[torch.storage.TypedStorage(dtype=torch.int64, device=cpu) of size 9]


# Contiguous란?

https://dsaint31.tistory.com/538#Continuous%--%EC%--%--%--Contiguous%--%EC%-D%--%--%EC%B-%A-%EC%-D%B--

In [ ]:
a.is_contiguous()

True

In [ ]:
a_sub = a[1:,0]
print(a_sub)

tensor([3, 6])


In [ ]:
print(a_sub.storage())
a_sub.is_contiguous()

 0
 1
 2
 3
 4
 5
 6
 7
 8
[torch.storage.TypedStorage(dtype=torch.int64, device=cpu) of size 9]


False

In [ ]:
a.storage().data_ptr() == a_sub.storage().data_ptr()

True

In [ ]:
a_t = torch.transpose(a,0,1)
a_t

tensor([[0, 3, 6],
        [1, 4, 7],
        [2, 5, 8]])

In [ ]:
print(a_t.storage())
a_t.is_contiguous()

 0
 1
 2
 3
 4
 5
 6
 7
 8
[torch.storage.TypedStorage(dtype=torch.int64, device=cpu) of size 9]


False

In [ ]:
a_t[0,1] = 999
print(a_t)
print('-------')
print(a)
print('-------')
print(a_sub)

tensor([[  0, 999,   6],
        [  1,   4,   7],
        [  2,   5,   8]])
-------
tensor([[  0,   1,   2],
        [999,   4,   5],
        [  6,   7,   8]])
-------
tensor([999,   6])


In [ ]:
print(f"{a.shape=}, {a.size()=}, {a.stride()=}, {a.storage_offset()=}")
print(f"{a_t.shape=}, {a_t.size()=}, {a_t.stride()=}, {a_t.storage_offset()=}")
print(f"{a_sub.shape=}, {a_sub.size()=}, {a_sub.stride()=}, {a_sub.storage_offset()=}")


a.shape=torch.Size([3, 3]), a.size()=torch.Size([3, 3]), a.stride()=(3, 1), a.storage_offset()=0
a_t.shape=torch.Size([3, 3]), a_t.size()=torch.Size([3, 3]), a_t.stride()=(1, 3), a_t.storage_offset()=0
a_sub.shape=torch.Size([2]), a_sub.size()=torch.Size([2]), a_sub.stride()=(3,), a_sub.storage_offset()=3


In [ ]:
a_v = a.view((9,))
a_v

tensor([  0,   1,   2, 999,   4,   5,   6,   7,   8])

In [ ]:
a_r = a.reshape((9,))
a_r[3] = 777
print('데이터 저장공간을 share함.')
print('a_r: af.reshape')
print(a_r)
print('a: orginal')
print(a)
print('a_t: transposed')
print(a_t)

데이터 저장공간을 share함.
a_r: af.reshape
tensor([  0,   1,   2, 777,   4,   5,   6,   7,   8])
a: orginal
tensor([[  0,   1,   2],
        [777,   4,   5],
        [  6,   7,   8]])
a_t: transposed
tensor([[  0, 777,   6],
        [  1,   4,   7],
        [  2,   5,   8]])


In [ ]:
a_t.view((9,)) # not working

RuntimeError: view size is not compatible with input tensor's size and stride (at least one dimension spans across two contiguous subspaces). Use .reshape(...) instead.

In [ ]:
a_t_r = a_t.reshape((9,)) # uncontiguous tensor에서는 새로 생성이 이루어짐.
a_t_r

tensor([  0, 777,   6,   1,   4,   7,   2,   5,   8])

In [ ]:
a_t_r[1] = 999
print(a_t_r)
print(a_t) # 달라짐.

tensor([  0, 999,   6,   1,   4,   7,   2,   5,   8])
tensor([[  0, 777,   6],
        [  1,   4,   7],
        [  2,   5,   8]])


In [ ]:
act = a_t.contiguous()
act

tensor([[  0, 777,   6],
        [  1,   4,   7],
        [  2,   5,   8]])

In [ ]:
act.storage()

 0
 777
 6
 1
 4
 7
 2
 5
 8
[torch.storage.TypedStorage(dtype=torch.int64, device=cpu) of size 9]

In [ ]:
act[1] = 999
print(act)
print(a_t)

tensor([[  0, 777,   6],
        [999, 999, 999],
        [  2,   5,   8]])
tensor([[  0, 777,   6],
        [  1,   4,   7],
        [  2,   5,   8]])


# permutation (순열)

> 인용구 추가



* `permute`: 축의 순서를 바꿈.

In [ ]:
print(a)
print('------')
print('permute axis-0 and axis-1 = transpose')
ap = a.permute([1,0]) # permute axis-0 and axis-1
ap

tensor([[  0,   1,   2],
        [777,   4,   5],
        [  6,   7,   8]])
------
permute axis-0 and axis-1 = transpose


tensor([[  0, 777,   6],
        [  1,   4,   7],
        [  2,   5,   8]])

In [ ]:
a.storage().data_ptr() == ap.storage().data_ptr()

True

In [ ]:
print(ap.is_contiguous())

False


In [ ]:
ap2 = ap.contiguous()
a.storage().data_ptr() == ap2.storage().data_ptr()

False

In [ ]:
ap2.is_contiguous()

True

In [ ]:
a

tensor([[  0,   1,   2],
        [777,   4,   5],
        [  6,   7,   8]])

In [ ]:
ap

tensor([[  0, 777,   6],
        [  1,   4,   7],
        [  2,   5,   8]])

In [ ]:
ap2

tensor([[  0, 777,   6],
        [  1,   4,   7],
        [  2,   5,   8]])